In [1]:
# Importing the necessary libraries and packages
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import time

# Reading in the excel files
onet_codes = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/ONET Code Descriptions.xlsx')
states = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/List of States.xlsx')
#proxy = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Proxy Servers - Confirmed.xlsx', header=None)

# Creating list of variables for pulling all jobs by ONET Code and State"
onet_codes_list = onet_codes['ONET Code']
states_list = states['Abbreviation']
#proxy_list = ['154.58.202.47:1337', '186.121.235.66:8080', '35.236.207.242:33333', '209.79.65.132:8080', '45.152.188.241:3128', '118.69.111.51:8080', '103.249.621:4443']
pages = list(range(1,6))

# Scrapping the Data Synchronously

### Pulling Job Postings

In [ ]:
# Setting the url for the first website
url = 'https://www.careeronestop.org/Credentials/Toolkit/find-jobs-results.aspx?keyword=11-1011.00&location=IN&source=NLX&currentpage=1&pagesize=500'

# Setting the header to the same as the main domain name to avoid response code 403
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Edg/115.0.1901.203'}

# Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Edg/115.0.1901.203
# Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 Edg/116.0.1938.62

# Confirming the connection is good
# 200 - connected
# 202 - accepted, but no connected
# 403 - forbidden
response = requests.get(url, headers=headers)
print(response.status_code)

200


In [ ]:
states_list[48:49]

48    WI
Name: Abbreviation, dtype: object

In [ ]:
# Pulling the postings

list_of_jobs = []
errors = []
#counter = 0

start = time.time()

for code in onet_codes_list:
  for state in states_list[48:49]:
    for page in pages:
      try:
        url = f'https://www.careeronestop.org/Credentials/Toolkit/find-jobs-results.aspx?keyword={code}&location={state}&source=NLX&currentpage={page}&pagesize=500'
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Edg/115.0.1901.203'}
        print('Working on code {} page {}'.format(code, page))

        # Pulling the webpage
        response = requests.get(url, headers=headers,
                                #proxies={'https': proxy_list[counter], 'http': proxy_list[counter]}
                                )

        # Formating the page into html content
        soup = bs(response.content, 'html.parser')

        # Finding the table that contains information we're looking for
        table = soup.find('table', class_ = 'cos-table-responsive')

        # Obtaining the column headers and adding them to data frame
        table_titles = soup.find_all('th')
        table_headers = [title.text.strip() for title in table_titles]
        df = pd.DataFrame(columns = table_headers)

        # Obtaining the row data for each column
        column_data = table.find_all('tr')

        for row in column_data[1:-1]:
          row_data = row.find_all('td')
          individual_row_data = [data.text.strip() for data in row_data]

          length = len(df)
          df.loc[length] = individual_row_data

        # Assigning Onet_code to list of jobs
        onet_code = url[81:91]
        df['Onet Code'] = onet_code

        # Cleaning the column titled Company
        df['Company'] = df['Company'].map(lambda x: x.replace('\r\n',''))

        # Reducing the size of the date frame
        categories = ['Onet Code', 'Job Title', 'Company', 'Location']
        date_time = ['Date Posted']

        df[categories] = df[categories].astype('category')
        df[date_time] = df[date_time].astype('datetime64[ns]')

        # Rearranging the column order
        df = df[['Onet Code', 'Job Title', 'Company', 'Location', 'Date Posted']]

        # Sending scrapped data to outside list
        list_of_jobs.append(df)
      except:
        errors.append(url)
      finally:
        try:
          for url in errors:
            print('re-trying code {} page {}'.format(code, page))

            # Pulling the webpage
            response = requests.get(url, headers=headers,
                                #proxies={'https': proxy_list[counter], 'http': proxy_list[counter]}
                                )

            # Formating the page into html content
            soup = bs(response.content, 'html.parser')

            # Finding the table that contains information we're looking for
            table = soup.find('table', class_ = 'cos-table-responsive')

            # Obtaining the column headers and adding them to data frame
            table_titles = soup.find_all('th')
            table_headers = [title.text.strip() for title in table_titles]
            df = pd.DataFrame(columns = table_headers)

            # Obtaining the row data for each column
            column_data = table.find_all('tr')

            for row in column_data[1:-1]:
              row_data = row.find_all('td')
              individual_row_data = [data.text.strip() for data in row_data]

              length = len(df)
              df.loc[length] = individual_row_data

            # Assigning Onet_code to list of jobs
            onet_code = url[81:91]
            df['Onet Code'] = onet_code

            # Cleaning the column titled Company
            df['Company'] = df['Company'].map(lambda x: x.replace('\r\n',''))

            # Reducing the size of the date frame
            categories = ['Onet Code', 'Job Title', 'Company', 'Location']
            date_time = ['Date Posted']

            df[categories] = df[categories].astype('category')
            df[date_time] = df[date_time].astype('datetime64[ns]')

            # Rearranging the column order
            df = df[['Onet Code', 'Job Title', 'Company', 'Location', 'Date Posted']]

            # Sending scrapped data to outside list
            list_of_jobs.append(df)

            errors.remove(url)
        except:
          pass
      # Rotating the proxy server
      #counter += 1
      #counter % len(proxy_list)

end = time.time()
total_time = end - start
print('It took {} seconds to complete data scrapping'.format(total_time))
print('There were {} urls that could not be run'.format(len(errors)))

jobs_df = pd.concat(list_of_jobs,ignore_index=True)

Streaming output truncated to the last 5000 lines.
Working on code 11-3051.01 page 5
Working on code 11-3051.02 page 1
Working on code 11-3051.02 page 2
Working on code 11-3051.02 page 3
Working on code 11-3051.02 page 4
Working on code 11-3051.02 page 5
Working on code 11-3051.03 page 1
Working on code 11-3051.03 page 2
Working on code 11-3051.03 page 3
Working on code 11-3051.03 page 4
Working on code 11-3051.03 page 5
Working on code 11-3051.04 page 1
Working on code 11-3051.04 page 2
Working on code 11-3051.04 page 3
Working on code 11-3051.04 page 4
Working on code 11-3051.04 page 5
Working on code 11-3051.06 page 1
Working on code 11-3051.06 page 2
Working on code 11-3051.06 page 3
Working on code 11-3051.06 page 4
Working on code 11-3051.06 page 5
Working on code 11-3061.00 page 1
Working on code 11-3061.00 page 2
Working on code 11-3061.00 page 3
Working on code 11-3061.00 page 4
Working on code 11-3061.00 page 5
Working on code 11-3071.00 page 1
Working on code 11-3071.00 page

In [ ]:
# Confirming the results
jobs_df

,Onet Code,Job Title,Company,Location,Date Posted
0,11-1011.00,Senior VP Chief Clinical Officer,QUARTZ HEALTH ...,"MADISON, WI",2023-06-20
1,11-1011.00,Chief Clinical Officer,Evolent Health ...,"Madison, WI",2023-08-26
2,11-1011.00,Senior Vice President of Development,University of Wisconsin Foundation ...,"Madison, WI",2023-08-27
3,11-1011.00,Workforce Health Medical Director North,"FROEDTERT HEALTH, INC ...","MENOMONEE FALLS, WI",2023-08-08
4,11-1011.00,Vice President Chief Development,MCW: SARGEANT HEALTH CENTER ...,"MILWAUKEE, WI",2023-08-12
...,...,...,...,...,...
150438,53-7199.00,Heavy Equipment Operator,"Tradesmen International, Inc. ...","EAU CLAIRE, WI",2022-09-22
150439,53-7199.00,Heavy Equipment Operator,"Tradesmen International, Inc. ...","ONALASKA, WI",2022-09-22
150440,53-7199.00,Material Handler (Weekends),Prime360 ...,"Oconomowoc, WI",2020-12-18
150441,55-2012.00,Deputy Register in Probate,Walworth County ...,"Elkhorn, WI",2023-05-22


In [ ]:
jobs_df.to_excel('drive/MyDrive/Colab Notebooks/Wisconsin_job_listings.xlsx', index=False)

## Pulling Wage Information

In [ ]:
url = 'https://www.onetonline.org/link/localwages/11-1011.00?st=TX'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Edg/115.0.1901.203'}

# confirming the connection is good
response = requests.get(url, headers=headers)
response

<Response [200]>

In [ ]:
# Running the loop synchronously

list_of_wages = []
errors = []

start = time.time()

for code in onet_codes_list:
  for state in states_list:
    try:
      url = f'https://www.onetonline.org/link/localwages/{code}?st={state}'
      headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Edg/115.0.1901.203'}
      print('Working on code {} for {}'.format(code, state))

      # confirming the connection is good
      response = requests.get(url, headers=headers)

      # Finding the table the contains the desired information
      soup = bs(response.content, 'html.parser')
      table = soup.find('table', class_ = 'table tablesorter tablesorter-bootstrap table-responsive-md w-100 table-fluid')

      # Obtaining the column headers and adding them to the data frame
      table_titles = soup.find_all('th')
      table_headers = [title.text.strip() for title in table_titles]
      df = pd.DataFrame(columns = table_headers[:6])

      # Obtaining the data for each column
      column_data = table.find_all('tr')

      for row in column_data[1:]:
        row_data = row.find_all('td')
        individual_row_data = [data.text.strip() for data in row_data]

        length = len(df)
        df.loc[length] = individual_row_data

      # Fixing formatting on wage columns and converting to integer dtype
      column_names = list(df)[1:]

      for name in column_names:
        df[name] = df[name].map(lambda x: x.strip('$').rstrip('+').replace(',',""))
        df[name] = df[name].astype(float)

      # Assigning the Onet Code to the list of wages
      onet_code = url[43:53]
      df['Onet Code'] = onet_code
      df[['Onet Code', 'Location']] =df[['Onet Code', 'Location']].astype('category')

      # Rearranging the Column Order
      df = df[['Onet Code', 'Location', 'Annual Low (10%)', 'Annual QL (25%)', 'Annual Median (50%)', 'Annual QU (75%)', 'Annual High (90%)']]

      # Sending scrapped data to outside list
      list_of_wages.append(df)

    except:
      errors.append(url)
    finally:
      try:
        for url in errors:
          print('Re-trying on code {} for {}'.format(code, state))

          # confirming the connection is good
          response = requests.get(url, headers=headers)

          # Finding the table the contains the desired information
          soup = bs(response.content, 'html.parser')
          table = soup.find('table', class_ = 'table tablesorter tablesorter-bootstrap table-responsive-md w-100 table-fluid')

          # Obtaining the column headers and adding them to the data frame
          table_titles = soup.find_all('th')
          table_headers = [title.text.strip() for title in table_titles]
          df = pd.DataFrame(columns = table_headers[:6])

          # Obtaining the data for each column
          column_data = table.find_all('tr')

          for row in column_data[1:]:
            row_data = row.find_all('td')
            individual_row_data = [data.text.strip() for data in row_data]

            length = len(df)
            df.loc[length] = individual_row_data

          # Fixing formatting on wage columns and converting to integer dtype
          column_names = list(df)[1:]

          for name in column_names:
            df[name] = df[name].map(lambda x: x.strip('$').rstrip('+').replace(',',""))
            df[name] = df[name].astype(float)

          # Assigning the Onet Code to the list of wages
          onet_code = url[43:53]
          df['Onet Code'] = onet_code
          df[['Onet Code', 'Location']] =df[['Onet Code', 'Location']].astype('category')

          # Rearranging the Column Order
          df = df[['Onet Code', 'Location', 'Annual Low (10%)', 'Annual QL (25%)', 'Annual Median (50%)', 'Annual QU (75%)', 'Annual High (90%)']]

          # Sending scrapped data to outside list
          list_of_wages.append(df)

          errors.remove(url)
      except:
        pass

end = time.time()
total_time = end - start
print('It took {} seconds to complete data scrapping'.format(total_time))
print('There were {} urls that could not be run'.format(len(errors)))

wages_df = pd.concat(list_of_wages, ignore_index=True)

Streaming output truncated to the last 5000 lines.
Working on code 53-5021.00 for AK
Re-trying on code 53-5021.00 for AK
Working on code 53-5021.00 for AL
Re-trying on code 53-5021.00 for AL
Working on code 53-5021.00 for AR
Re-trying on code 53-5021.00 for AR
Working on code 53-5021.00 for AZ
Re-trying on code 53-5021.00 for AZ
Working on code 53-5021.00 for CA
Re-trying on code 53-5021.00 for CA
Working on code 53-5021.00 for CO
Re-trying on code 53-5021.00 for CO
Working on code 53-5021.00 for CT
Re-trying on code 53-5021.00 for CT
Working on code 53-5021.00 for DC
Re-trying on code 53-5021.00 for DC
Working on code 53-5021.00 for DE
Re-trying on code 53-5021.00 for DE
Working on code 53-5021.00 for FL
Re-trying on code 53-5021.00 for FL
Working on code 53-5021.00 for GA
Re-trying on code 53-5021.00 for GA
Working on code 53-5021.00 for HI
Re-trying on code 53-5021.00 for HI
Working on code 53-5021.00 for IA
Re-trying on code 53-5021.00 for IA
Working on code 53-5021.00 for ID
Re-tr

In [ ]:
# Confirming the results
wages_df

,Onet Code,Location,Annual Low (10%),Annual QL (25%),Annual Median (50%),Annual QU (75%),Annual High (90%)
0,11-1011.00,United States,74920.0,122480.0,189520.0,239200.0,239200.0
1,11-1011.00,Alaska,72680.0,114420.0,148720.0,217310.0,239200.0
2,11-1011.00,Alaska nonmetropolitan area,70620.0,122320.0,136470.0,212550.0,239200.0
3,11-1011.00,"Anchorage, AK",74400.0,110470.0,149080.0,224990.0,239200.0
4,11-1011.00,"Fairbanks, AK",64560.0,102670.0,166070.0,209710.0,239200.0
...,...,...,...,...,...,...,...
367799,53-7199.00,Northwestern Wisconsin nonmetropolitan area,24840.0,27050.0,28510.0,31200.0,45570.0
367800,53-7199.00,South Central Wisconsin nonmetropolitan area,26390.0,26450.0,28120.0,31280.0,35010.0
367801,53-7199.00,Western Wisconsin nonmetropolitan area,27050.0,27050.0,27050.0,28930.0,53460.0
367802,53-7199.00,United States,30070.0,33280.0,38800.0,48840.0,60070.0


In [ ]:
wages_df.to_excel('drive/MyDrive/Colab Notebooks/wage_statistics.xlsx', index=False)

## Pulling Job Employment Trends

In [ ]:
url = 'https://www.onetonline.org/link/localtrends/11-1011.00?st=TX'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Edg/115.0.1901.203'}

response = requests.get(url, headers=headers)
response

<Response [200]>

In [ ]:
# Running the loop synchronously

list_of_trends = []
errors = []

start = time.time()

for code in onet_codes_list:
  for state in states_list:
      try:
        url = f'https://www.onetonline.org/link/localtrends/{code}?st={state}'
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Edg/115.0.1901.203'}
        print('Working on code {} for {}'.format(code, state))

        response = requests.get(url, headers=headers)

        soup = bs(response.content, 'html.parser')

        table = soup.find('div', class_ ="col-12 col-sm-8 col-md-6")

        table_titles = table.find_all('dt')
        table_headers = [title.text.strip() for title in table_titles]
        df = pd.DataFrame(columns = table_headers)

        # Obtaining the raw data for each column
        # We are not initiating a for loop since their is only one row of data
        column_data = table.find_all('dd')
        row_data = [data.text.strip() for data in column_data]
        length = len(df)
        df.loc[length] = row_data

        # Adding Onet and state information
        onet_code = url[44:54]
        state = url[58:60]

        df['Onet Code'] = onet_code
        df['State'] = state
        df[['Onet Code', 'State']] = df[['Onet Code', 'State']].astype('category')
        df = df[['Onet Code', 'State', 'Employment (2020)', 'Projected employment (2030)', 'Projected growth (2020-2030)', 'Projected annual job openings (2020-2030)']]

        # Fixing formatting on Employment and Growth Columns and coverting to a float dtype
        column_names = list(df)[2:]

        for name in column_names:
          df[name] = df[name].map(lambda x: x.rstrip(' employees').rstrip('%').replace(',',''))
          df[name] = df[name].astype(float)

        # Sending scrapped data to outside list
        list_of_trends.append(df)

      except:
        errors.append(url)

end = time.time()
total_time = end - start
print('It took {} seconds to complete data scrapping'.format(total_time))
print('There were {} urls that could not be run'.format(len(errors)))

trends_df = pd.concat(list_of_trends, ignore_index=True)

Streaming output truncated to the last 5000 lines.
Working on code 51-9081.00 for AK
Working on code 51-9081.00 for AL
Working on code 51-9081.00 for AR
Working on code 51-9081.00 for AZ
Working on code 51-9081.00 for CA
Working on code 51-9081.00 for CO
Working on code 51-9081.00 for CT
Working on code 51-9081.00 for DC
Working on code 51-9081.00 for DE
Working on code 51-9081.00 for FL
Working on code 51-9081.00 for GA
Working on code 51-9081.00 for HI
Working on code 51-9081.00 for IA
Working on code 51-9081.00 for ID
Working on code 51-9081.00 for IL
Working on code 51-9081.00 for IN
Working on code 51-9081.00 for KS
Working on code 51-9081.00 for KY
Working on code 51-9081.00 for LA
Working on code 51-9081.00 for MA
Working on code 51-9081.00 for MD
Working on code 51-9081.00 for ME
Working on code 51-9081.00 for MI
Working on code 51-9081.00 for MN
Working on code 51-9081.00 for MO
Working on code 51-9081.00 for MS
Working on code 51-9081.00 for MT
Working on code 51-9081.00 for 

In [ ]:
# Confirming the results
trends_df

,Onet Code,State,Employment (2020),Projected employment (2030),Projected growth (2020-2030),Projected annual job openings (2020-2030)
0,11-1011.00,AK,1590.0,1770.0,11.0,130.0
1,11-1011.00,AL,2120.0,1950.0,-8.0,120.0
2,11-1011.00,AR,3140.0,3140.0,0.0,210.0
3,11-1011.00,AZ,4990.0,5320.0,7.0,380.0
4,11-1011.00,CA,47400.0,47100.0,-1.0,3170.0
...,...,...,...,...,...,...
42908,53-7199.00,UT,100.0,120.0,20.0,10.0
42909,53-7199.00,VA,310.0,330.0,7.0,40.0
42910,53-7199.00,VT,70.0,70.0,0.0,10.0
42911,53-7199.00,WA,880.0,970.0,10.0,130.0


In [ ]:
trends_df.to_excel('drive/MyDrive/Colab Notebooks/trends.xlsx', index=False)

# Asynchronous Code

In [ ]:
!pip install aiolimiter
#!pip install nest-asyncio

In [ ]:
import nest_asyncio
nest_asyncio.apply()
from aiolimiter import AsyncLimiter
import asyncio
import aiohttp

In [ ]:
# Setting the url for the first website
url = 'https://www.careeronestop.org/Credentials/Toolkit/find-jobs-results.aspx?keyword=11-1011.00&location=Texas&source=NLX&currentpage=1&pagesize=500'

# Setting the header to the same as the main domain name to avoid response code 403
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Edg/115.0.1901.203'}

# Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Edg/115.0.1901.203 (Original Version)
# Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 Edg/116.0.1938.62 (New version?)

# Confirming the connection is good
# 200 - connected
# 202 - accepted, but no connected
# 403 - forbidden
response = requests.get(url, headers=headers)
response

<Response [200]>

Currently, due to the massive volume of jobs, we are limiting our scrapping to Texas. While running the pulls asynchronously is a massive improvment in performance (The synchronous code would take 72 hours), due to the API limit of 300 calls per minute, it would take 14 hours to pull all the data. Since google co-labs kills the session after 12 hours, it is not feasible to pull this data.

It may be possible to pull all the data with multi-threading or tricking the API into thinking the calls are coming from multiple places. However, I have deemed that optimization out of the scope for this project.

In [ ]:
# Creating the urls
def get_urls():
  urls = []
  for code in onet_codes_list:
    for state in states_list[43:44]:
      for page in pages:
        url = f'https://www.careeronestop.org/Credentials/Toolkit/find-jobs-results.aspx?keyword={code}&location={state}&source=NLX&currentpage={page}&pagesize=500'
        urls.append(url)
  return urls

# Scrapping the html code
async def get_html(url, limiter):
  headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Edg/115.0.1901.203'}
  async with limiter:
    async with aiohttp.ClientSession() as session:
      async with session.get(url, headers=headers, ssl=False) as response:
        return await response.text()

async def clean_data(response):
  soup = bs(response, 'html.parser')

  # Finding the table that contains information we're looking for
  table = soup.find('table', class_ = 'cos-table-responsive')

  # Obtaining the column headers and adding them to data frame
  table_titles = soup.find_all('th')
  table_headers = [title.text.strip() for title in table_titles]
  df = pd.DataFrame(columns = table_headers)

  # Obtaining the row data for each column
  column_data = table.find_all('tr')

  for row in column_data[1:-1]:
    row_data = row.find_all('td')
    individual_row_data = [data.text.strip() for data in row_data]

    length = len(df)
    df.loc[length] = individual_row_data

  # Assigning Onet_code to list of jobs
  onet_code = url[81:91]
  df['Onet Code'] = onet_code

  # Cleaning the column titled Company
  df['Company'] = df['Company'].map(lambda x: x.replace('\r\n',''))

  # Reducing the size of the date frame
  categories = ['Onet Code', 'Job Title', 'Company', 'Location']
  date_time = ['Date Posted']

  df[categories] = df[categories].astype('category')
  df[date_time] = df[date_time].astype('datetime64[ns]')

  # Rearranging the column order
  df = df[['Onet Code', 'Job Title', 'Company', 'Location', 'Date Posted']]

  return df

# Putting it all together and adding the data cleaning
async def get_job_postings():

  list_of_jobs = []
  errors = []
  responses = []
  rate_limit = AsyncLimiter(300)

  urls = get_urls()
  for url in urls:
    response = await get_html(url, rate_limit)
    responses.append(response)
  try:
    for response in responses:
      df = await clean_data(response)
      responses.remove(response)

      # Sending scrapped data to list_of_jobs
      list_of_jobs.append(df)
  except:
    errors.append(response)

  # Retrieving list of scrapped data and converting to a dataframe
  jobs_df = pd.concat(list_of_jobs, ignore_index=True)
  print('There were {} urls that could not be run'.format(len(errors)))
  return (jobs_df, errors)

In [ ]:
jobs_df, errors = asyncio.run(get_job_postings())
jobs_df

# Putting All the Job Postings Together

In [ ]:
alabama = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/alabama_job_listings.xlsx')
alabama['State - Long'] = 'alabama'
alaska = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/alaska_job_listings.xlsx')
alaska['State - Long'] = 'alaska'
arizona = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/arizona_job_listings.xlsx')
arizona['State - Long'] = 'arizona'
arkansas = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/arkansas_job_listings.xlsx')
arkansas['State - Long'] = 'arkansas'
california = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/california_job_listings.xlsx')
california['State - Long'] = 'california'
colorado = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/colorado_job_listings.xlsx')
colorado['State - Long'] = 'colorado'
connecticut = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/connecticut_job_listings.xlsx')
connecticut['State - Long'] = 'connecticut'
delaware = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/Delaware_job_listings.xlsx')
delaware['State - Long'] = 'delaware'
district_of_columbia = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/District_of_Columbia_job_listings.xlsx')
district_of_columbia['State - Long'] = 'district of columbia'
florida = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/Florida_job_listings.xlsx')
florida['State - Long'] = 'florida'
georgia = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/Georgia_job_listings.xlsx')
georgia['State - Long'] = 'georgia'
hawaii = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/Hawaii_job_listings.xlsx')
hawaii['State - Long'] = 'hawaii'
idaho = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/Idaho_job_listings.xlsx')
idaho['State - Long'] = 'idaho'
illinois = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/Illinois_job_listings.xlsx')
illinois['State - Long'] = 'illinois'
indiana = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/Indiana_job_listings.xlsx')
indiana['State - Long'] = 'indiana'
iowa = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/Iowa_job_listings.xlsx')
iowa['State - Long'] = 'iowa'
kansas = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/Kansas_job_listings.xlsx')
kansas['State - Long'] = 'kansas'
kentucky = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/Kentucky_job_listings.xlsx')
kentucky['State - Long'] = 'kentucky'
louisiana = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/Louisiana_job_listings.xlsx')
louisiana['State - Long'] = 'louisana'
maine = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/Maine_job_listings.xlsx')
maine['State - Long'] = 'maine'
maryland = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/Maryland_job_listings.xlsx')
maryland['State - Long'] = 'maryland'
massachusetts = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/Massachusetts_job_listings.xlsx')
massachusetts['State - Long'] = 'massachusetts'
michigan = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/Michigan_job_listings.xlsx')
michigan['State - Long'] = 'michigan'
minnesota = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/Minnesota_job_listings.xlsx')
minnesota['State - Long'] = 'minnesota'
mississippi = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/Mississippi_job_listings.xlsx')
mississippi['State - Long'] = 'mississippi'
missouri = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/Missouri_job_listings.xlsx')
missouri['State - Long'] = 'missouri'
montana = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/Montana_job_listings.xlsx')
montana['State - Long'] = 'montana'
nebraska = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/Nebraska_job_listings.xlsx')
nebraska['State - Long'] = 'nebraska'
nevada = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/Nevada_job_listings.xlsx')
nevada['State - Long'] = 'nevada'
new_hampshire = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/New_Hampshire_job_listings.xlsx')
new_hampshire['State - Long'] = 'new hampshire'
new_jersey = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/New_Jersey_job_listings.xlsx')
new_jersey['State - Long'] = 'new jersey'
new_mexico = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/New_Mexico_job_listings.xlsx')
new_mexico['State - Long'] = 'new mexico'
new_york = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/New_York_job_listings.xlsx')
new_york['State - Long'] = 'new_ ork'
north_carolina = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/North_Carolina_job_listings.xlsx')
north_carolina['State - Long'] = 'north carolina'
north_dakota = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/North_Dakota_job_listings.xlsx')
north_dakota['State - Long'] = 'north dakota'
ohio = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/Ohio_job_listings.xlsx')
ohio['State - Long'] = 'ohio'
oklahoma = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/Oklahoma_job_listings.xlsx')
oklahoma['State - Long'] = 'oklahoma'
oregon = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/Oregon_job_listings.xlsx')
oregon['State - Long'] = 'oregon'
pennsylvania = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/Pennsylvannia_job_listings.xlsx')
pennsylvania['State - Long'] = 'pennsylvania'
rhode_island = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/Rhode_Island_job_listings.xlsx')
rhode_island['State - Long'] = 'rhode_island'
south_carolina = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/South_Carolina_job_listings.xlsx')
south_carolina['State - Long'] = 'south_carolina'
south_dakota = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/South_Dakota_job_listings.xlsx')
south_dakota['State - Long'] = 'south_dakota'
tennessee = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/Tennessee_job_listings.xlsx')
tennessee['State - Long'] = 'tennessee'
texas = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/Texas_job_listings.xlsx')
texas['State - Long'] = 'texas'
utah = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/Utah_job_listings.xlsx')
utah['State - Long'] = 'utah'
vermont = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/Vermont_job_listings.xlsx')
vermont['State - Long'] = 'vermont'
virginia = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/Virginia_job_listings.xlsx')
virginia['State - Long'] = 'virginia'
washington = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/Washington_job_listings.xlsx')
washington['State - Long'] = 'washington'
west_virginia = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/West_Virginia_job_listings.xlsx')
west_virginia['State - Long'] = 'west virginia'
wisconsin = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/Wisconsin_job_listings.xlsx')
wisconsin['State - Long'] = 'wisconsin'
wyoming = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/State Job Listings/Wyoming_job_listings.xlsx')
wyoming['State - Long'] = 'wyoming'

In [ ]:
list_of_jobs = []
list_of_jobs.append(alabama)
list_of_jobs.append(alaska)
list_of_jobs.append(arizona)
list_of_jobs.append(arkansas)
list_of_jobs.append(california)
list_of_jobs.append(colorado)
list_of_jobs.append(connecticut)
list_of_jobs.append(district_of_columbia)
list_of_jobs.append(delaware)
list_of_jobs.append(florida)
list_of_jobs.append(georgia)
list_of_jobs.append(hawaii)
list_of_jobs.append(iowa)
list_of_jobs.append(idaho)
list_of_jobs.append(illinois)
list_of_jobs.append(indiana)
list_of_jobs.append(kansas)
list_of_jobs.append(kentucky)
list_of_jobs.append(louisiana)
list_of_jobs.append(massachusetts)
list_of_jobs.append(maryland)
list_of_jobs.append(maine)
list_of_jobs.append(michigan)
list_of_jobs.append(minnesota)
list_of_jobs.append(missouri)
list_of_jobs.append(mississippi)
list_of_jobs.append(montana)
list_of_jobs.append(north_carolina)
list_of_jobs.append(north_dakota)
list_of_jobs.append(nebraska)
list_of_jobs.append(new_hampshire)
list_of_jobs.append(new_jersey)
list_of_jobs.append(new_mexico)
list_of_jobs.append(nevada)
list_of_jobs.append(new_york)
list_of_jobs.append(ohio)
list_of_jobs.append(oklahoma)
list_of_jobs.append(oregon)
list_of_jobs.append(pennsylvania)
list_of_jobs.append(rhode_island)
list_of_jobs.append(south_carolina)
list_of_jobs.append(south_dakota)
list_of_jobs.append(tennessee)
list_of_jobs.append(texas)
list_of_jobs.append(utah)
list_of_jobs.append(virginia)
list_of_jobs.append(vermont)
list_of_jobs.append(washington)
list_of_jobs.append(wisconsin)
list_of_jobs.append(west_virginia)
list_of_jobs.append(wyoming)

In [ ]:
jobs_df_1 = pd.concat(list_of_jobs[0:8],ignore_index=True)
jobs_df_2 = pd.concat(list_of_jobs[8:16],ignore_index=True)
jobs_df_3 = pd.concat(list_of_jobs[16:24],ignore_index=True)
jobs_df_4 = pd.concat(list_of_jobs[24:32],ignore_index=True)
jobs_df_5 = pd.concat(list_of_jobs[32:40],ignore_index=True)
jobs_df_6 = pd.concat(list_of_jobs[40:45],ignore_index=True)
jobs_df_7 = pd.concat(list_of_jobs[45:52],ignore_index=True)

In [ ]:
x = jobs_df_1

errors = []

# Extracting the city from the location column
x['Location'] = x['Location'].astype('string')
locations = x['Location'].tolist()
split_locations = []

for location in locations:
  try:
    location_split = location.split(',')
    split_locations.append(location_split)
  except:
    errors.append(location)

i = x['Location'].loc[x['Location'].isin(errors)].index
x.drop(i, inplace=True)

jobs_df_1 = x

In [ ]:
x = jobs_df_7

errors = []

# Extracting the city from the location column
x['Location'] = x['Location'].astype('string')
locations = x['Location'].tolist()
split_locations = []

for location in locations:
  try:
    location_split = location.split(',')
    split_locations.append(location_split)
  except:
    errors.append(location)

i = x['Location'].loc[x['Location'].isin(errors)].index
x.drop(i, inplace=True)

jobs_df_7 = x

In [ ]:
x = jobs_df_1

# Removes Job Title Column
x.drop("Job Title", axis=1, inplace=True)

# Cleans the company column
x['Company'] = x['Company'].map(lambda x: x.replace('\n',''))
x['Company'] = x['Company'].map(lambda x: x.replace('Federal Contractor',''))
company_dirty = x['Company'].tolist()
company_clean = [x.strip() for x in company_dirty]

# Replacing the copmany column with cleaned version
x.drop("Company", axis=1, inplace=True)
x["Company"] = company_clean

# Extracting the city from the location column
x['Location'] = x['Location'].astype('string')
locations = x['Location'].tolist()
split_locations = []

for location in locations:
  location_split = location.split(',')
  split_locations.append(location_split)

city = [item[0] for item in split_locations]

# Replacing location with the city
x.drop('Location', axis=1, inplace=True)
x['City'] = city

x = x[['Onet Code', 'Company', 'City', 'State - Long', 'Date Posted']]

jobs_df_1 = x

In [ ]:
x = jobs_df_2

# Removes Job Title Column
x.drop("Job Title", axis=1, inplace=True)

# Cleans the company column
x['Company'] = x['Company'].map(lambda x: x.replace('\n',''))
x['Company'] = x['Company'].map(lambda x: x.replace('Federal Contractor',''))
company_dirty = x['Company'].tolist()
company_clean = [x.strip() for x in company_dirty]

# Replacing the copmany column with cleaned version
x.drop("Company", axis=1, inplace=True)
x["Company"] = company_clean

# Extracting the city from the location column
x['Location'] = x['Location'].astype('string')
locations = x['Location'].tolist()
split_locations = []

for location in locations:
  location_split = location.split(',')
  split_locations.append(location_split)

city = [item[0] for item in split_locations]

# Replacing location with the city
x.drop('Location', axis=1, inplace=True)
x['City'] = city

x = x[['Onet Code', 'Company', 'City', 'State - Long', 'Date Posted']]

jobs_df_2 = x

In [ ]:
x = jobs_df_3

# Removes Job Title Column
x.drop("Job Title", axis=1, inplace=True)

# Cleans the company column
x['Company'] = x['Company'].map(lambda x: x.replace('\n',''))
x['Company'] = x['Company'].map(lambda x: x.replace('Federal Contractor',''))
company_dirty = x['Company'].tolist()
company_clean = [x.strip() for x in company_dirty]

# Replacing the copmany column with cleaned version
x.drop("Company", axis=1, inplace=True)
x["Company"] = company_clean

# Extracting the city from the location column
x['Location'] = x['Location'].astype('string')
locations = x['Location'].tolist()
split_locations = []

for location in locations:
  location_split = location.split(',')
  split_locations.append(location_split)

city = [item[0] for item in split_locations]

# Replacing location with the city
x.drop('Location', axis=1, inplace=True)
x['City'] = city

x = x[['Onet Code', 'Company', 'City', 'State - Long', 'Date Posted']]

jobs_df_3 = x

In [ ]:
x = jobs_df_4

# Removes Job Title Column
x.drop("Job Title", axis=1, inplace=True)

# Cleans the company column
x['Company'] = x['Company'].map(lambda x: x.replace('\n',''))
x['Company'] = x['Company'].map(lambda x: x.replace('Federal Contractor',''))
company_dirty = x['Company'].tolist()
company_clean = [x.strip() for x in company_dirty]

# Replacing the copmany column with cleaned version
x.drop("Company", axis=1, inplace=True)
x["Company"] = company_clean

# Extracting the city from the location column
x['Location'] = x['Location'].astype('string')
locations = x['Location'].tolist()
split_locations = []

for location in locations:
  location_split = location.split(',')
  split_locations.append(location_split)

city = [item[0] for item in split_locations]

# Replacing location with the city
x.drop('Location', axis=1, inplace=True)
x['City'] = city

x = x[['Onet Code', 'Company', 'City', 'State - Long', 'Date Posted']]

jobs_df_4 = x

In [ ]:
x = jobs_df_5

# Removes Job Title Column
x.drop("Job Title", axis=1, inplace=True)

# Cleans the company column
x['Company'] = x['Company'].map(lambda x: x.replace('\n',''))
x['Company'] = x['Company'].map(lambda x: x.replace('Federal Contractor',''))
company_dirty = x['Company'].tolist()
company_clean = [x.strip() for x in company_dirty]

# Replacing the copmany column with cleaned version
x.drop("Company", axis=1, inplace=True)
x["Company"] = company_clean

# Extracting the city from the location column
x['Location'] = x['Location'].astype('string')
locations = x['Location'].tolist()
split_locations = []

for location in locations:
  location_split = location.split(',')
  split_locations.append(location_split)

city = [item[0] for item in split_locations]

# Replacing location with the city
x.drop('Location', axis=1, inplace=True)
x['City'] = city

x = x[['Onet Code', 'Company', 'City', 'State - Long', 'Date Posted']]

jobs_df_5 = x

In [ ]:
x = jobs_df_6

# Removes Job Title Column
x.drop("Job Title", axis=1, inplace=True)

# Cleans the company column
x['Company'] = x['Company'].map(lambda x: x.replace('\n',''))
x['Company'] = x['Company'].map(lambda x: x.replace('Federal Contractor',''))
company_dirty = x['Company'].tolist()
company_clean = [x.strip() for x in company_dirty]

# Replacing the copmany column with cleaned version
x.drop("Company", axis=1, inplace=True)
x["Company"] = company_clean

# Extracting the city from the location column
x['Location'] = x['Location'].astype('string')
locations = x['Location'].tolist()
split_locations = []

for location in locations:
  location_split = location.split(',')
  split_locations.append(location_split)

city = [item[0] for item in split_locations]

# Replacing location with the city
x.drop('Location', axis=1, inplace=True)
x['City'] = city

x = x[['Onet Code', 'Company', 'City', 'State - Long', 'Date Posted']]

jobs_df_6 = x

In [ ]:
x = jobs_df_7

# Removes Job Title Column
x.drop("Job Title", axis=1, inplace=True)

# Cleans the company column
x['Company'] = x['Company'].map(lambda x: x.replace('\n',''))
x['Company'] = x['Company'].map(lambda x: x.replace('Federal Contractor',''))
company_dirty = x['Company'].tolist()
company_clean = [x.strip() for x in company_dirty]

# Replacing the copmany column with cleaned version
x.drop("Company", axis=1, inplace=True)
x["Company"] = company_clean

# Extracting the city from the location column
x['Location'] = x['Location'].astype('string')
locations = x['Location'].tolist()
split_locations = []

for location in locations:
  location_split = location.split(',')
  split_locations.append(location_split)

city = [item[0] for item in split_locations]

# Replacing location with the city
x.drop('Location', axis=1, inplace=True)
x['City'] = city

x = x[['Onet Code', 'Company', 'City', 'State - Long', 'Date Posted']]

jobs_df_7 = x

In [ ]:
jobs_df_1.to_excel('drive/MyDrive/Colab Notebooks/Job_listings_1.xlsx', index=False)
jobs_df_2.to_excel('drive/MyDrive/Colab Notebooks/Job_listings_2.xlsx', index=False)
jobs_df_3.to_excel('drive/MyDrive/Colab Notebooks/Job_listings_3.xlsx', index=False)
jobs_df_4.to_excel('drive/MyDrive/Colab Notebooks/Job_listings_4.xlsx', index=False)
jobs_df_5.to_excel('drive/MyDrive/Colab Notebooks/Job_listings_5.xlsx', index=False)
jobs_df_6.to_excel('drive/MyDrive/Colab Notebooks/Job_listings_6.xlsx', index=False)
jobs_df_7.to_excel('drive/MyDrive/Colab Notebooks/Job_listings_7.xlsx', index=False)

# Creating the Master List

In [2]:
jobs_df_1 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Job_listings_1.xlsx')
jobs_df_2 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Job_listings_2.xlsx')
jobs_df_3 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Job_listings_3.xlsx')
jobs_df_4 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Job_listings_4.xlsx')
jobs_df_5 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Job_listings_5.xlsx')
jobs_df_6 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Job_listings_6.xlsx')
jobs_df_7 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Job_listings_7.xlsx')


with pd.ExcelWriter('drive/MyDrive/Colab Notebooks/Job Listings - Master.xlsx') as writer:
  jobs_df_1.to_excel(writer, sheet_name="master_list_1")
  jobs_df_2.to_excel(writer, sheet_name="master_list_2")
  jobs_df_3.to_excel(writer, sheet_name="master_list_3")
  jobs_df_4.to_excel(writer, sheet_name="master_list_4")
  jobs_df_5.to_excel(writer, sheet_name="master_list_5")
  jobs_df_6.to_excel(writer, sheet_name="master_list_6")
  jobs_df_7.to_excel(writer, sheet_name="master_list_7")